# Homework 5

In [1]:
# imports
import numpy as np
from importlib import reload

from oceanpy import constants
import unyt
from oceanpy.forces import coriolis as ofc

from bokeh import plotting
from bokeh import models

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
def set_fontsize(p, fsz):
    '''
    Parameters
    ----------
    ax : Bokeh plot class
    fsz : float
      Font size
    '''
    p.xaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.xaxis.major_label_text_font_size = "{:d}pt".format(fsz)
    #
    p.yaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.yaxis.major_label_text_font_size = "{:d}pt".format(fsz)

# 1. Surface Mixed Layer

## a) Properties of the surface mixing layer

### Depth, density, salinity, temperature, buoyancy

## b) Why does it occur?

### It is generated by two key processes:  (i) wind forcing, i.e. shear; (ii) buoyancy changes due to heat flux and the addition of fresh water or evaporation.

## c) When+where?

### I suspect everywhere, but am guessing there are figures in Chapter 4 or 7 (not the Supp) showing there are special places.  From (b) above, areas where there are substnatial buyouancy changes would be relevent, e.g. near river mouths.  And where winds are shifting.

# 2. Wind 

### Note:  I don't have access to Talley Section 5.8 but have read Stewart

## a) Wind vs. wind stress

### Wind is the transport of air in the atmosphere.  Wind stress is the horizontal force of the wind on the sea surface due to the frictional interaction between the two.  It may also be conceptualized as the flow of momentrum from wind to ocean.

## b) Wind stress in the Pacific

### Looking at Fig 4.4 of Stewart, in the middle Pacific in July the winds are weakly to the West near the Equator and travel to the West with increasing velocity as one increases in latitute until $\approx 45^\circ$ where they begin to reverse.

### The behavior to the South is decreasing speed to the West that reverses and becomes fast to the East below 30S.  

### Wind stress is proportional to wind sea-surface speed.

## c) $\partial \tau_x / \partial y$

### At 30N, the winds are changing from to the West ($-x$) to the East $+x$.  This means $\partial \tau_x / \partial y > 0$.

### At 40S, the winds are to the East ($+x$) but decrease with $y$, so $\partial \tau_x / \partial y < 0$.


## d) Wind stress curl

### Northern Hemisphere

#### Positive:  Very near the poles.  Upper Pacific and Atlantic oceans

#### Negative: At mid-latitudes, i.e. $\sim 30$N

### Southern Hemisphere

#### At 40S in each of the Pacific, Atlantic and Indian oceans

----

# 3. Ekman Solution

## Northern wind with $\tau_y = 0.1 \, \rm N \, m^{-2}$ in Santa Cruz ($\phi = 36.9^\circ$)

## a) Plot of $u(z), v(z)$ with $A_z = 0.01 \, \rm m^2 \, s^{-1}$

In [3]:
def uofz(z, Ty=0.1*unyt.N/unyt.m**2, Az=0.01*unyt.m**2/unyt.s, phi=36.9*unyt.deg):
    # Calculate the constants
    f = ofc.coriolis_parameter(phi)
    V0 = Ty / np.sqrt(unyt.density_seawater**2 * f * Az)
    a = np.sqrt(f / 2 / Az)
    #
    u = V0 * np.exp(a*z) * np.sin(np.pi/4 - a*z)
    return u

In [14]:
def vofz(z, Ty=0.1*unyt.N/unyt.m**2, Az=0.01*unyt.m**2/unyt.s, phi=36.9*unyt.deg):
    # Calculate the constants
    f = ofc.coriolis_parameter(phi)
    V0 = Ty / np.sqrt(unyt.density_seawater**2 * f * Az)
    a = np.sqrt(f / 2 / Az)
    #
    v = V0 * np.exp(a*z) * np.cos(np.pi/4 - a*z)
    return v

In [15]:
z = np.linspace(0., -90, 100)*unyt.m
u = uofz(z)
v = vofz(z)
u[0:10].to('m/s')

unyt_array([0.07382187, 0.0735653 , 0.07283651, 0.07169399, 0.07019242,
            0.06838273, 0.06631219, 0.06402447, 0.06155978, 0.05895497], 'm/s')

In [17]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Velocity with depth', x_axis_label='z [m]',
                   y_axis_label='u,v [m/s]', x_range=(0,-90))

p.line(z, u, legend='u', line_width=2)
p.line(z, v, legend='v', line_width=2, color='red')


p.legend.location = "top_right"

set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### At the surface, the current is $45^\circ$ to the right of the wind, i.e. towards the NE

## b) $U$

### The equation is simply:  $U = V_0 \exp(az)$

In [22]:
def Uofz(z, Ty=0.1*unyt.N/unyt.m**2, Az=0.01*unyt.m**2/unyt.s, phi=36.9*unyt.deg, verbose=False):
    # Calculate the constants
    f = ofc.coriolis_parameter(phi)
    V0 = Ty / np.sqrt(unyt.density_seawater**2 * f * Az)
    a = np.sqrt(f / 2 / Az)
    if verbose:
        print(V0, a.to('1/m'))
    #
    U = V0 * np.exp(a*z)
    return U

In [23]:
U = Uofz(z, verbose=True)

0.007792635340763262 N/(Omega_earth*density_seawater*m**3) 0.06616901423759869 1/m


In [25]:
1./0.06617

15.112588786459119

### There is a $1/e$ reduction at $az=1$.  This occurs at approximately 15m.

## c) Modify $A_z = 0.0001 \, \rm m^2 \, s^{-1}$ (molecular viscosity)

In [26]:
U = Uofz(z, verbose=True, Az=0.0001*unyt.m**2/unyt.s)

0.07792635340763263 N/(Omega_earth*density_seawater*m**3) 0.6616901423759869 1/m


### Lowering $A_z$ by 100 increases $a$ by 10 and the e-folding depth is reduced by the same factor, i.e. 1.5m

### To affect tens of meters, eddy viscosity must be the one.

## d) $M_E$

## Eddy

In [27]:
def calcMEx(Ty, phi=36.9*unyt.deg):
    f = ofc.coriolis_parameter(phi)
    MEx = Ty / f
    return MEx

In [29]:
MEx = calcMEx(0.1*unyt.N/unyt.m**2)
MEx.to('kg/m/s')

unyt_quantity(1141.98572418, 'kg/(m*s)')

### The direction is positive x, i.e. East

### This quantity is independent of $A_z$

In [31]:
def calcQx(Ty, Y, phi=36.9*unyt.deg):
    Qx = calcMEx(Ty, phi=phi) / unyt.density_seawater
    return Qx.to('m**2 / s')

In [33]:
Qx = calcQx(0.1*unyt.N/unyt.m**2, 1*unyt.m)
Qx

unyt_quantity(1.11565624, 'm**2/s')

## e) The eddy viscosity only sets the characteristic depth of the transport

----

# 3. Turbulence

## a) Turbulence affects flows over distances of a few cm and therefore leads to mixing on these scales.

## b) Three factors:  surface cooling, wind stress, wave breaking

----

In [3]:
constants.oceandata2019.density_seawater

unyt_quantity(1023.6, 'kg/m**3')

In [9]:
(5*unyt.density_seawater).to('kg/m**3')

unyt_quantity(5118., 'kg/m**3')